In [79]:
import tensorflow as tf
import numpy as np
import healpy as hp

from DeepSphere import healpy_networks as hp_nn
from DeepSphere import gnn_layers
import matplotlib.pyplot as plt

NSIDE = 32
NPIX = hp.nside2npix(NSIDE)

all_ipix = list(range(NPIX))
hp_map = np.arange(NPIX)

hp_map_nest = hp.reorder(hp_map, r2n=True)
print(np.shape(hp_map_nest))


# define some layers (can be mixed with normal tf layers)
# define only one output layer --> only want the mean !?
layers = [gnn_layers.HealpyPseudoConv(p=1, Fout=4),
          gnn_layers.HealpyPool(p=1),
          hp_nn.HealpyChebyshev5(K=5, Fout=8),
          gnn_layers.HealpyPseudoConv(p=2, Fout=16),
          hp_nn.HealpyMonomial(K=5, Fout=32),
          hp_nn.Healpy_ResidualLayer("CHEBY", layer_kwargs={"K": 5}),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(32),
          tf.keras.layers.Dense(1)]

 # build a model
tf.keras.backend.clear_session()
indices = np.arange(NPIX)
model = hp_nn.HealpyGCNN(nside=NSIDE, indices=indices, layers=layers)

(12288,)
Detected a reduction factor of 16, the input with nside 32 will be reduced to 2 during a forward pass. Checking for consistency with indices...
indices seem consistent...


In [93]:
# we build the model
# unlike most CNN achitectures you MUST supply in the batch dimension here
model.build(input_shape=(1, len(indices), 1))
#model.summary(line_length=100)

inp = np.random.normal(loc=12, size=(1, len(indices), 1)).astype(np.float32)
out = model(inp)


model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), 
              loss='mean_absolute_error')


train_data = []
train_labels = []
for i in range(20000):
    mean = np.random.rand()
    train_labels.append(mean)
    train_data.append(np.random.normal(loc=mean, size=(len(indices), 1)).astype(np.float32))
train_data = np.asarray(train_data)
train_labels = np.asarray(train_labels)


model.fit(train_data, train_labels, epochs=3, validation_split=0.2)

Epoch 1/3
500/500 [==============================] - 9s 17ms/step - loss: 1.8345 - val_loss: 0.0285
Epoch 2/3
500/500 [==============================] - 9s 17ms/step - loss: 0.0303 - val_loss: 0.0304
Epoch 3/3
500/500 [==============================] - 9s 17ms/step - loss: 0.0250 - val_loss: 0.0230


In [95]:
for i in range(10):
    mean= np.random.rand()
    predictions = model(np.random.normal(loc=mean, size=(1, len(indices), 1)))
    print("Model predicted %f, true label is %s" % (predictions, mean))

Model predicted 0.502622, true label is 0.5206882829899127
Model predicted 0.785956, true label is 0.7686607154171029
Model predicted 0.784718, true label is 0.764076150093772
Model predicted 0.099775, true label is 0.11185919866242022
Model predicted 0.853114, true label is 0.8277204293537743
Model predicted 0.972826, true label is 0.9518457810144277
Model predicted 0.440336, true label is 0.45067530400212197
Model predicted 0.238425, true label is 0.25359721773312693
Model predicted 0.349898, true label is 0.3463078541882406
Model predicted 0.568584, true label is 0.5626299243717059
